## <b><font color='darkblue'>What are function tools?</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/), [Parent page](https://github.com/johnklee/ml_articles/blob/master/google/agent_development_kit/Tools.ipynb)) <font size='3ptx'><b>When out-of-the-box tools don't fully meet specific requirements, developers can create custom function tools.</b> This allows for <b>tailored functionality</b>, such as connecting to proprietary databases or implementing unique algorithms.</font>

For example, a function tool, "`myfinancetool`", might be a function that calculates a specific financial metric. <b>ADK also supports long running functions, so if that calculation takes a while, the agent can continue working on other tasks</b>.

ADK offers several ways to create functions tools, each suited to different levels of complexity and control::
1. <b><font size='3ptx'>[Function Tool](#1.-Function-Tool)</font></b>
2. <b><font size='3ptx'>[Long Running Function Tool](#2.-Long-Running-Function-Tool)</font></b>
3. Agents-as-a-Tool

In [1]:
from IPython.display import display, Markdown, Latex
import warnings


warnings.filterwarnings('ignore')

def show_source_code(src_path: str):
    source_code = !cat $src_path
    display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

## <b><font color='darkblue'>1. Function Tool</font></b>
<font size='3ptx'><b>Transforming a Python function into a tool is a straightforward way to integrate custom logic into your agents.</b> When you assign a function to an agent’s tools list, the framework automatically wraps it as a <b><font color='blue'>FunctionTool</font></b>.</font> 

### <b><font color='darkgreen'>How it Works</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#how-it-works)) <font size='3ptx'><b>The ADK framework automatically inspects your Python function's signature—including its name, docstring, parameters, type hints, and default values—to generate a schema.</b> This schema is what the LLM uses to understand the tool's purpose, when to use it, and what arguments it requires.</font>

### <b><font color='darkgreen'>Defining Function Signatures</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#defining-function-signatures)) <b><font size='3ptx'>A well-defined function signature is crucial for the LLM to use your tool correctly.</font></b>

#### <b>Parameters</b>
You can define functions with required parameters, optional parameters, and variadic arguments. Here’s how each is handled:

<b>Required Parameters</b><br/>
A parameter is considered **required** if it has a type hint but **no default value**. The LLM must provide a value for this argument when it calls the tool. e.g.:
```python
def get_weather(city: str, unit: str):
    """
    Retrieves the weather for a city in the specified unit.

    Args:
        city (str): The city name.
        unit (str): The temperature unit, either 'Celsius' or 'Fahrenheit'.
    """
    # ... function logic ...
    return {"status": "success", "report": f"Weather for {city} is sunny."}
```

In this example, both `city` and `unit` are mandatory. If the LLM tries to call `get_weather` without one of them, the ADK will return an error to the LLM, prompting it to correct the call.

<b>Optional Parameters with Default Values</b><br/>
A parameter is considered **optional** if you **provide a default value**. This is the standard Python way to define optional arguments. The ADK correctly interprets these and does not list them in the `required` field of the tool schema sent to the LLM. e.g.:
```python
def search_flights(destination: str, departure_date: str, flexible_days: int = 0):
    """
    Searches for flights.

    Args:
        destination (str): The destination city.
        departure_date (str): The desired departure date.
        flexible_days (int, optional): Number of flexible days for the search. Defaults to 0.
    """
    # ... function logic ...
    if flexible_days > 0:
        return {"status": "success", "report": f"Found flexible flights to {destination}."}
    return {"status": "success", "report": f"Found flights to {destination} on {departure_date}."}
```

Here, `flexible_days` is optional. The LLM can choose to provide it, but it's not required.

<b>Optional Parameters with `typing.Optional`</b><br/>
You can also mark a parameter as optional using `typing.Optional[SomeType]` or the `| None` syntax (<font color='brown'>Python 3.10+</font>). This signals that the parameter can be `None`. When combined with a default value of `None`, it behaves as a standard optional parameter. e.g.:
```python
rom typing import Optional

def create_user_profile(username: str, bio: Optional[str] = None):
    """
    Creates a new user profile.

    Args:
        username (str): The user's unique username.
        bio (str, optional): A short biography for the user. Defaults to None.
    """
    # ... function logic ...
    if bio:
        return {"status": "success", "message": f"Profile for {username} created with a bio."}
    return {"status": "success", "message": f"Profile for {username} created."}
```

<b>Variadic Parameters (*args and **kwargs)</b><br/>
While you can include `*args` (<font color='brown'>variable positional arguments</font>) and `**kwargs` (<font color='brown'>variable keyword arguments</font>) in your function signature for other purposes, <b>they are ignored by the ADK framework when generating the tool schema for the LLM. The LLM will not be aware of them and cannot pass arguments to them</b>. It's best to rely on explicitly defined parameters for all data you expect from the LLM.

#### <b>Return Type</b>
<font size='3ptx'><b>The preferred return type for a Function Tool is a dictionary (or `dict`).</b> This allows you to structure the response with key-value pairs, providing context and clarity to the LLM. If your function returns a type other than a dictionary, the framework automatically wraps it into a dictionary with a single key named "`result`".</font>

<b><font size='3ptx'>Strive to make your return values as descriptive as possible</font></b>. For example, instead of returning a numeric error code, return a dictionary with an "error_message" key containing a human-readable explanation. Remember that <b>the LLM, not a piece of code, needs to understand the result. As a best practice, include a "status" key in your return dictionary to indicate the overall outcome</b> (<font color='brown'>e.g., "success", "error", "pending"</font>), providing the LLM with a clear signal about the operation's state.

#### <b>Docstrings</b>
<font size='3ptx'><b>The docstring of your function serves as the tool's description and is sent to the LLM.</b> Therefore, a well-written and comprehensive docstring is crucial for the LLM to understand how to use the tool effectively. Clearly explain the purpose of the function, the meaning of its parameters, and the expected return values.</font>

### <b><font color='darkgreen'>Example</font></b>
Below example will require installation of package [**`yfinance`**](https://github.com/ranaroussi/yfinance):

In [2]:
!pip show yfinance

Name: yfinance
Version: 0.2.65
Summary: Download market data from Yahoo! Finance API
Home-page: https://github.com/ranaroussi/yfinance
Author: Ran Aroussi
Author-email: ran@aroussi.com
License: Apache
Location: /usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.12/site-packages
Requires: beautifulsoup4, curl_cffi, frozendict, multitasking, numpy, pandas, peewee, platformdirs, protobuf, pytz, requests, websockets
Required-by: 


In [3]:
show_source_code('function_tools_example1.py')


```python
#!/bin/env python
import asyncio

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

import yfinance as yf


APP_NAME = "stock_app"
USER_ID = "1234"
SESSION_ID = "session1234"


def get_stock_price(symbol: str):
  """
  Retrieves the current stock price for a given symbol.

  Args:
    symbol (str): The stock symbol (e.g., "AAPL", "GOOG").

  Returns:
    float: The current stock price, or None if an error occurs.
  """
  try:
    stock = yf.Ticker(symbol)
    historical_data = stock.history(period="1d")
    if not historical_data.empty:
      current_price = historical_data['Close'].iloc[-1]
      return current_price
    return None
  except Exception as e:
    print(f"Error retrieving stock price for {symbol}: {e}")
    return None


stock_price_agent = Agent(
    model='gemini-2.0-flash',
    name='stock_agent',
    instruction=(
        'You are an agent who retrieves stock prices. If a ticker symbol is '
        'provided, fetch the current price. If only a company name is given, '
        'first perform a Google search to find the correct ticker symbol before'
        ' retrieving the stock price. If the provided ticker symbol is invalid '
        'or data cannot be retrieved, inform the user that the stock price '
        'could not be found.'),
    description=(
        'This agent specializes in retrieving real-time stock prices. Given a '
        'stock ticker symbol (e.g., AAPL, GOOG, MSFT) or the stock name, use '
        'the tools and reliable data sources to provide the most up-to-date '
        'price.'),
    # You can add Python functions directly to the tools list;
    # they will be automatically wrapped as FunctionTools.
    tools=[get_stock_price],
)


# Session and Runner
async def setup_session_and_runner():
  session_service = InMemorySessionService()
  session = await session_service.create_session(
      app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
  runner = Runner(
      agent=stock_price_agent,
      app_name=APP_NAME,
      session_service=session_service)
  return session, runner


# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  session, runner = await setup_session_and_runner()
  events = runner.run_async(
      user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  async for event in events:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)


if __name__ == '__main__':
  asyncio.run(call_agent_async('stock price of GOOG'))
```

In [4]:
import function_tools_example1

await function_tools_example1.call_agent_async('stock price of GOOG')

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Agent Response:  The current stock price for GOOG is 201.63.



### <b><font color='darkgreen'>Best Practices</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#best-practices)) <b><font size='3ptx'>While you have considerable flexibility in defining your function, remember that simplicity enhances usability for the LLM.</font></b>

Consider these guidelines:
* <b><font size='3ptx'>Fewer Parameters are Better</font></b>: Minimize the number of parameters to reduce complexity.
* <b><font size='3ptx'>Simple Data Types</font></b>: Favor primitive data types like str and int over custom classes whenever possible.
* <b><font size='3ptx'>Meaningful Names</font></b>: The function's name and parameter names significantly influence how the LLM interprets and utilizes the tool. Choose names that clearly reflect the function's purpose and the meaning of its inputs. Avoid generic names like `do_stuff()` or `beAgent()`.

## <b><font color='darkblue'>2. Long Running Function Tool</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#2-long-running-function-tool)) <font size='3ptx'><b>Designed for tasks that require a significant amount of processing time without blocking the agent's execution.</b> This tool is a subclass of <font color='blue'><b>FunctionTool</b></font>.</font>

When using a <b><font color='blue'>LongRunningFunctionTool</font></b>, your function can initiate the long-running operation and optionally return an **initial result** (<font color='brown'>e.g. the long-running operation id</font>). Once a long running function tool is invoked the agent runner will pause the agent run and let the agent client to decide whether to continue or wait until the long-running operation finishes.

<b>The agent client can query the progress of the long-running operation and send back an intermediate or final response</b>. The agent can then continue with other tasks. An example is the human-in-the-loop scenario where the agent needs human approval before proceeding with a task.

### <b><font color='darkgreen'>How it Works</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#how-it-works_1)) <b><font size='3ptx'>In Python, you wrap a function with <font color='blue'>LongRunningFunctionTool</font>:</font></b>

1. <b><font size='3ptx'>Initiation</font></b>: When the LLM calls the tool, your function starts the long-running operation.
2. <b><font size='3ptx'>Initial Updates</font></b>: Your function should optionally return an initial result (<font color='brown'>e.g. the long-running operaiton id</font>). The ADK framework takes the result and sends it back to the LLM packaged within a <b><font color='blue'>FunctionResponse</font></b>. This allows the LLM to inform the user (<font color='brown'>e.g., status, percentage complete, messages</font>). And then the agent run is ended / paused.
3. <b><font size='3ptx'>Continue or Wait</font></b>: After each agent run is completed. Agent client can query the progress of the long-running operation and decide whether to continue the agent run with an intermediate response (<font color='brown'>to update the progress</font>) or wait until a final response is retrieved. Agent client should send the intermediate or final response back to the agent for the next run.
4. <b><font size='3ptx'>Framework Handling</font></b>: The ADK framework manages the execution. It sends the intermediate or final <b><font color='blue'>FunctionResponse</font></b> sent by agent client to the LLM to generate a user friendly message.

### <b><font color='darkgreen'>Creating the Tool</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#creating-the-tool)) <b><font size='3ptx'>Define your tool function and wrap it using the <font color='blue'>LongRunningFunctionTool</font> class:</font></b>

```python
# 1. Define the long running function
def ask_for_approval(
    purpose: str, amount: float
) -> dict[str, Any]:
    """Ask for approval for the reimbursement."""
    # create a ticket for the approval
    # Send a notification to the approver with the link of the ticket
    return {'status': 'pending', 'approver': 'Sean Zhou', 'purpose' : purpose, 'amount': amount, 'ticket-id': 'approval-ticket-1'}

def reimburse(purpose: str, amount: float) -> str:
    """Reimburse the amount of money to the employee."""
    # send the reimbrusement request to payment vendor
    return {'status': 'ok'}

# 2. Wrap the function with LongRunningFunctionTool
long_running_tool = LongRunningFunctionTool(func=ask_for_approval)
```

### <b><font color='darkgreen'>Intermediate / Final result Updates</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#intermediate-final-result-updates)) <font size='3ptx'><b>Agent client received an event with long running function calls and check the status of the ticket.</b> Then Agent client can send the intermediate or final response back to update the progress. The framework packages this value (<font color='brown'>even if it's None</font>) into the content of the <font color='blue'><b>FunctionResponse</b></font> sent back to the LLM. </font>

In [5]:
show_source_code('function_tools_example2.py ')


```python
#!/usr/bin/env python
import asyncio

from typing import Any
from google.adk.agents import Agent
from google.adk.events import Event
from google.adk.runners import Runner
from google.adk.tools import LongRunningFunctionTool
from google.adk.sessions import InMemorySessionService
from google.genai import types


APP_NAME = "human_in_the_loop"
USER_ID = "1234"
SESSION_ID = "session1234"


# 1. Define the long running function
def ask_for_approval(purpose: str, amount: float) -> dict[str, Any]:
  """Ask for approval for the reimbursement.

  Args:
    purpose: The purpose of the approval.
    amount: The amount for approval.

  Returns:
    dict object with keys:
    - status: The approval status.
    - approver: The people to ask for approval,
    - purpose: The copy of input `purpose`.
    - amount: The copy of input `amount`.
    - ticket-id: The id of fired ticket.
  """
  # create a ticket for the approval
  # Send a notification to the approver with the link of the ticket
  return {
      'status': 'pending',
      'approver': 'Sean Zhou',
      'purpose' : purpose,
      'amount': amount,
      'ticket-id': 'approval-ticket-1'
  }


def reimburse(purpose: str, amount: float) -> dict[str, Any]:
  """Reimburse the amount of money to the employee.

  Args:
    purpose: The purpose of the approval.
    amount: The amount for approval.

  Returns:
    dict object with key `status` as 'ok' or 'reject'.
  """
  # send the reimbrusement request to payment vendor
  return {'status': 'ok'}


# 2. Wrap the function with LongRunningFunctionTool
long_running_tool = LongRunningFunctionTool(func=ask_for_approval)

# 3. Use the tool in an Agent
file_processor_agent = Agent(
    # Use a model compatible with function calling
    model="gemini-2.0-flash",
    name='reimbursement_agent',
    instruction="""
      You are an agent whose job is to handle the reimbursement process for
      the employees. If the amount is less than $100, you will automatically
      approve the reimbursement.

      If the amount is greater than $100, you will
      ask for approval from the manager. If the manager approves, you will
      call reimburse() to reimburse the amount to the employee. If the manager
      rejects, you will inform the employee of the rejection.
    """,
    tools=[reimburse, long_running_tool])


# Session and Runner
async def setup_session_and_runner():
  print('Seting up session and runner...')
  session_service = InMemorySessionService()
  session = await session_service.create_session(
      app_name=APP_NAME,
      user_id=USER_ID,
      session_id=SESSION_ID)
  runner = Runner(
      agent=file_processor_agent,
      app_name=APP_NAME,
      session_service=session_service)
  return session, runner


# Agent Interaction
async def call_agent_async(query: str, session, runner):
  def get_long_running_function_call(event: Event) -> types.FunctionCall:
    # Get the long running function call from the event
    if (
        not event.long_running_tool_ids
        or not event.content
        or not event.content.parts
    ):
      return

    for part in event.content.parts:
      if (
          part
          and part.function_call
          and event.long_running_tool_ids
          and part.function_call.id in event.long_running_tool_ids
      ):
        return part.function_call

  def get_function_response(
      event: Event, function_call_id: str) -> types.FunctionResponse:
    # Get the function response for the fuction call with specified id.
    if not event.content or not event.content.parts:
      return

    for part in event.content.parts:
      if (
          part
          and part.function_response
          and part.function_response.id == function_call_id
      ):
        return part.function_response

  if session is None or runner is None:
    session, runner = await setup_session_and_runner()

  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run_async(
      user_id=USER_ID,
      session_id=SESSION_ID,
      new_message=content)

  print("\nRunning agent...")
  events_async = runner.run_async(
      session_id=session.id, user_id=USER_ID, new_message=content)

  long_running_function_call, long_running_function_response, ticket_id = (
      None, None, None)

  async for event in events_async:
    # Use helper to check for the specific auth request event
    if not long_running_function_call:
      long_running_function_call = get_long_running_function_call(event)
      if long_running_function_call:
        print(
            '\tGetting long running function call: '
            f'{long_running_function_call}')
    elif long_running_function_call and not long_running_function_response:
      long_running_function_response = get_function_response(
          event, long_running_function_call.id)
      print(
          '\tGetting long running function resp with id '
          f'"{long_running_function_call.id}": '
          f'{long_running_function_response}')
      if long_running_function_response:
        ticket_id = long_running_function_response.response['ticket-id']

    if event.content and event.content.parts:
      text = ''.join(
          part.text or ''
          for part in event.content.parts
          if hasattr(part, 'text')
      )
      if text:
        print(f'[{event.author}]: {text}')

  if long_running_function_response:
    # query the status of the correpsonding ticket via tciket_id
    # send back an intermediate / final response
    updated_response = long_running_function_response.model_copy(deep=True)
    updated_response.response = {'status': 'approved'}
    print('Sending approved event...')
    async for event in runner.run_async(
        session_id=session.id,
        user_id=USER_ID,
        new_message=types.Content(
            parts=[
                types.Part(function_response = updated_response)
            ], role='user')
    ):
      if event.content and event.content.parts:
        text = ''.join(
            part.text or ''
            for part in event.content.parts
            if hasattr(part, 'text')
        )
        if text:
          print(f'[{event.author}]: {text}')

    long_running_function_call, long_running_function_response, ticket_id = (
        None, None, None)
  else:
    print('No need to wait for long running function resp.')

  return session, runner


if __name__ == '__main__':
  session, runner = None, None
  session, runner = asyncio.run(call_agent_async("Please reimburse 50$ for meals", session, runner))
  asyncio.run(call_agent_async("Please reimburse 200$ for meals", session, runner))
```

In [6]:
import function_tools_example2

session, runner = None, None

In [7]:
session, runner = await function_tools_example2.call_agent_async("Please reimburse 50$ for meals", session, runner) 

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Seting up session and runner...

Running agent...


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


[reimbursement_agent]: OK. I have reimbursed $50 for meals.

No need to wait for long running function resp.


In [8]:
session, runner = await function_tools_example2.call_agent_async("Please reimburse 200$ for meals", session, runner)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



Running agent...


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


	Getting long running function call: id='adk-54fd2752-fd5b-4c36-bf5a-f9ae256d0d60' args={'purpose': 'meals', 'amount': 200} name='ask_for_approval'
	Getting long running function resp with id "adk-54fd2752-fd5b-4c36-bf5a-f9ae256d0d60": will_continue=None scheduling=None id='adk-54fd2752-fd5b-4c36-bf5a-f9ae256d0d60' name='ask_for_approval' response={'status': 'pending', 'approver': 'Sean Zhou', 'purpose': 'meals', 'amount': 200, 'ticket-id': 'approval-ticket-1'}


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


[reimbursement_agent]: OK. I have asked for approval from the manager for $200 for meals. I'll let you know the decision.

Sending approved event...


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


[reimbursement_agent]: OK. I have reimbursed $200 for meals.



Key aspects of this example:
* <b><font size='3ptx' color='blue'>LongRunningFunctionTool</font></b>: Wraps the supplied method/function; the framework handles sending yielded updates and the final return value as sequential FunctionResponses.
* <b><font size='3ptx'>Agent instruction</font></b>: Directs the LLM to use the tool and understand the incoming <font color='blue'><b>FunctionResponse</b></font> stream (<font color='brown'>progress vs. completion</font>) for user updates.
* <b><font size='3ptx'>Final return</font></b>: The function returns the final result dictionary, which is sent in the concluding <font color='blue'><b>FunctionResponse</b></font> to indicate completion.

## <b><font color='darkblue'>3. Agent-as-a-Tool</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#3-agent-as-a-tool)) <font size='3ptx'><b>This powerful feature allows you to leverage the capabilities of other agents within your system by calling them as tools.</b> The Agent-as-a-Tool enables you to invoke another agent to perform a specific task, effectively <b>delegating responsibility</b>. This is conceptually similar to creating a Python function that calls another agent and uses the agent's response as the function's return value.</font>

### <b><font color='darkgreen'>Key difference from sub-agents</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#key-difference-from-sub-agents)) It's important to distinguish an Agent-as-a-Tool from a Sub-Agent:
* <b><font size='3ptx'>Agent-as-a-Tool</font></b>: When Agent `A` calls Agent `B` as a tool (<font color='brown'>using Agent-as-a-Tool</font>), Agent `B`'s answer is passed back to Agent `A`, which then summarizes the answer and generates a response to the user. Agent `A` retains control and continues to handle future user input.
* <b><font size='3ptx'>Sub-agent</font></b>: When Agent `A` calls Agent `B` as a sub-agent, the responsibility of answering the user is completely transferred to Agent `B`. Agent `A` is effectively out of the loop. All subsequent user input will be answered by Agent `B`.

### <b><font color='darkgreen'>Usage</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#usage)) <b><font size='3ptx'>To use an agent as a tool, wrap the agent with the <font color='blue'>AgentTool</font> class</font></b>:
```python
tools=[AgentTool(agent=agent_b)]
```

### <b><font color='darkgreen'>Customization</font></b>
([source](https://google.github.io/adk-docs/tools/function-tools/#customization)) <b><font size='3ptx'>The <font color='blue'>AgentTool</font> class provides the following attributes for customizing its behavior</font></b>:
* <b>`skip_summarization: bool`:</b> If set to `True`, the framework will bypass the LLM-based summarization of the tool agent's response. <b>This can be useful when the tool's response is already well-formatted and requires no further processing</b>.

### <b><font color='darkgreen'>Example</font></b>

In [9]:
show_source_code('function_tools_example3.py')


```python
#!/usr/bin/env python
import asyncio

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.agent_tool import AgentTool
from google.genai import types


APP_NAME="summary_agent"
USER_ID="user1234"
SESSION_ID="1234"

summary_agent = Agent(
    model="gemini-2.0-flash",
    name="summary_agent",
    instruction=(
        "You are an expert summarizer. Please read the following text and "
        "provide a concise summary."),
    description="Agent to summarize text",
)

root_agent = Agent(
    model='gemini-2.0-flash',
    name='root_agent',
    instruction=(
        "You are a helpful assistant. When the user provides a text, use the "
        "'summarize' tool to generate a summary. Always forward the user's "
        "message exactly as received to the 'summarize' tool, without modifying"
        " or summarizing it yourself. Present the response from the tool to the"
        " user."),
    tools=[AgentTool(agent=summary_agent)]
)

# Session and Runner
async def setup_session_and_runner():
  session_service = InMemorySessionService()
  session = await session_service.create_session(
      app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
  runner = Runner(
      agent=root_agent, app_name=APP_NAME, session_service=session_service)
  return session, runner


# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  session, runner = await setup_session_and_runner()
  events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  async for event in events:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)


if __name__ == '__main__':
  long_text = """
  Quantum computing represents a fundamentally different approach to computation,
  leveraging the bizarre principles of quantum mechanics to process information.
  Unlike classical computers that rely on bits representing either 0 or 1,
  quantum computers use qubits which can exist in a state of superposition -
  effectively being 0, 1, or a combination of both simultaneously. Furthermore,
  qubits can become entangled, meaning their fates are intertwined regardless of
  distance, allowing for complex correlations. This parallelism and
  interconnectedness grant quantum computers the potential to solve specific types
  of incredibly complex problems - such as drug discovery, materials science,
  complex system optimization, and breaking certain types of cryptography - far
  faster than even the most powerful classical supercomputers could ever achieve,
  although the technology is still largely in its developmental stages."""
  asyncio.run(call_agent_async(long_text))
```

In [2]:
import function_tools_example3

long_text = """
A large language model (LLM) is a language model trained with self-supervised machine learning on a vast amount of text,
designed for natural language processing tasks, especially language generation.

The largest and most capable LLMs are generative pretrained transformers (GPTs),
which are largely used in generative chatbots such as ChatGPT, Gemini or Claude.
LLMs can be fine-tuned for specific tasks or guided by prompt engineering.
These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora,
but they also inherit inaccuracies and biases present in the data they are trained in.
"""

await function_tools_example3.call_agent_async(long_text)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Agent Response:  [Summary] Large language models (LLMs) are trained on vast amounts of text using self-supervised learning for NLP tasks, particularly language generation. Generative pretrained transformers (GPTs) are the largest and most capable LLMs, used in generative chatbots and can be fine-tuned or guided by prompts. While LLMs acquire predictive power regarding language, they also inherit inaccuracies and biases from their training data.



### <b><font color='darkgreen'>How it works</font></b>
1. When the `main_agent` receives the long text, its instruction tells it to use the '`summarize`' tool for long texts.
2. The framework recognizes '`summarize`' as an <b><font color='blue'>AgentTool</font></b> that wraps the `summary_agent`.
3. Behind the scenes, the `main_agent` will call the `summary_agent` with the long text as input.
4. The `summary_agent` will process the text according to its instruction and generate a summary.
5. <b>The response from the `summary_agent` is then passed back to the `main_agent`</b>.
6. The `main_agent` can then take the summary and formulate its final response to the user (<font color='brown'>e.g., "Here's a summary of the text: ..."</font>)